In [13]:
import numpy as np
from numpy.random import RandomState
from recommend.pmf import PMF
from recommend.utils.evaluation import RMSE
import pandas as pd
from sklearn import preprocessing
rand_state = RandomState(0)

In [3]:
df=pd.read_csv("../github-recommender/fdf.csv")

In [5]:
le = preprocessing.LabelEncoder()
le.fit(df.user)
df.user=le.transform(df.user)
le.fit(df.repo)
df.repo=le.transform(df.repo)

In [6]:
ratings=df.values

In [7]:
n_user=df.user.max()+1
n_item=df.repo.max()+1

In [8]:
print("num of user: %d" % n_user)
print("num of item: %d" % n_item)

num of user: 150662
num of item: 32464


In [9]:
# split data to training & testing
train_pct = 0.9
rand_state.shuffle(ratings)
train_size = int(train_pct * ratings.shape[0])
train = ratings[:train_size]
validation = ratings[train_size:]

print("training size: %d" % train.shape[0])
print("validation size: %d" % validation.shape[0])

training size: 720663
validation size: 80074


In [10]:
%%time
n_feature = 10
eval_iters = 20
print("training PMF model...")
pmf = PMF(n_user=n_user, n_item=n_item, n_feature=n_feature,
          epsilon=25., max_rating=1., min_rating=None, seed=0)
pmf.fit(train, n_iters=eval_iters)

training PMF model...
CPU times: user 43min 38s, sys: 588 ms, total: 43min 39s
Wall time: 43min 39s


In [11]:
pmf_train_preds = pmf.predict(train[:, :2])
pmf_train_rmse = RMSE(pmf_train_preds, train[:, 2])
pmf_val_preds = pmf.predict(validation[:, :2])
pmf_val_rmse = RMSE(pmf_val_preds, validation[:, 2])
print("PMF training RMSE %.3f, validation RMSE %.3f" % (pmf_train_rmse, pmf_val_rmse))

PMF training RMSE 0.000, validation RMSE 0.000


In [14]:
user_id = 0

pred_items = np.array([[user_id, i] for i in range(10)])
# pmf
pmf_pred = pmf.predict(pred_items)

print("PMF prediction for user %d" % user_id)
for i in range(10):
    print("item %d: pmf: %.3f" % (i, pmf_pred[i]))

PMF prediction for user 0
item 0: pmf: 1.000
item 1: pmf: 1.000
item 2: pmf: 1.000
item 3: pmf: 1.000
item 4: pmf: 1.000
item 5: pmf: 1.000
item 6: pmf: 1.000
item 7: pmf: 1.000
item 8: pmf: 1.000
item 9: pmf: 1.000
